In [1]:
import pandas as pd
import conn
import dataio

In [3]:
df = dataio.pull_cms_master()

In [3]:
df

,CrimeCase,arraign_date,arrest_date,closed_date,complaint_date,custody_date,filed_dist_date,juvenile_detained_date,open_date,time_to_file,...,case_is_aggravated_assault,case_is_armed_robbery,case_is_rape,case_has_rebuttable_presumption,case_trajectory,case_disposition,case_reason,case_is_juvenile,case_is_violent,case_is_difficult_victim
0,12dabc25-c71b-420a-96f4-00039dfb52b3,NaT,NaT,2019-06-04 00:00:00,NaT,NaT,NaT,NaT,2019-02-27,NaN,...,False,False,False,False,Prosecution Declined,Prosecution Declined,Statute of Limitations or Age of Case,False,True,False
1,cec83662-24da-4292-bd24-00049f333b24,2019-03-21,2019-03-20,2019-04-30 00:00:00,2019-03-20,NaT,2019-04-16,NaT,2019-03-21,26.0,...,False,False,False,False,Prosecuted,Prosecutor Dismissed,Insufficient Evidence,False,False,False
2,cec83662-24da-4292-bd24-00049f333b24,2019-03-21,2019-03-20,2019-04-30 00:00:00,2019-03-20,NaT,2019-04-16,NaT,2019-03-21,26.0,...,False,False,False,False,Prosecuted,Prosecutor Dismissed,Insufficient Evidence,False,False,False
3,cec83662-24da-4292-bd24-00049f333b24,2019-03-21,2019-03-20,2019-04-30 00:00:00,2019-03-20,NaT,2019-04-16,NaT,2019-03-21,26.0,...,False,False,False,False,Prosecuted,Prosecutor Dismissed,Insufficient Evidence,False,False,False
4,cec83662-24da-4292-bd24-00049f333b24,2019-03-21,2019-03-20,2019-04-30 00:00:00,2019-03-20,NaT,2019-04-16,NaT,2019-03-21,26.0,...,False,False,False,False,Prosecuted,Prosecutor Dismissed,Insufficient Evidence,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68429,d09f70dd-f8e4-4657-a58c-fff4cfefaec3,NaT,2010-11-29,2012-01-04 13:52:00,2010-11-29,NaT,NaT,NaT,2010-12-01,NaN,...,False,False,False,False,Prosecution Declined,Prosecution Declined,Law Enforcement Failed to Appear or Uncooperative,False,False,False
68430,d09f70dd-f8e4-4657-a58c-fff4cfefaec3,NaT,2010-11-29,2012-01-04 13:52:00,2010-11-29,NaT,NaT,NaT,2010-12-01,NaN,...,False,False,False,False,Prosecution Declined,Prosecution Declined,Law Enforcement Failed to Appear or Uncooperative,False,False,False
68431,d09f70dd-f8e4-4657-a58c-fff4cfefaec3,NaT,2010-11-29,2012-01-04 13:52:00,2010-11-29,NaT,NaT,NaT,2010-12-01,NaN,...,False,False,False,False,Prosecution Declined,Prosecution Declined,Law Enforcement Failed to Appear or Uncooperative,False,False,False
68432,0d61efa3-5c9a-46ca-930c-fff896ae393b,2021-01-25,2021-01-21,2021-03-10 00:00:00,2021-01-21,NaT,NaT,NaT,2021-01-25,NaN,...,False,False,False,False,Prosecution Declined,Prosecution Declined,Insufficient Evidence,False,False,False


In [20]:
%%time
#df2=pd.read_csv('stanford_data_pull', parse_dates=False, low_memory=False)
df2 = df.sort_values(by=['cms_defendant_guid', 'open_date'], ignore_index=True)

CPU times: total: 40.6 s
Wall time: 40.7 s


In [22]:
df2.shape

(1761970, 165)

In [ ]:
df2['CrimeCase'][df2.charge_sequence == 1].value_counts()

In [26]:
df2.drop([774153, 774155], inplace=True)

In [40]:
%%time
flag_df = df2[df2.charge_sequence==1].reset_index(drop=True)
flag_df['prior_arrest'] = False
flag_df['prior_indictment'] = False
flag_df['prior_conviction'] = False
flag_df['prior_violent_arrest'] = False
flag_df['prior_violent_indictment'] = False
flag_df['prior_violent_conviction'] = False
flag_df['prior_felony_arrest'] = False
flag_df['prior_felony_indictment'] = False
flag_df['prior_felony_conviction'] = False

CPU times: total: 14.5 s
Wall time: 14.5 s


In [42]:
%%time
priors_labels = ['prior_arrest', 'prior_indictment', 'prior_conviction',
                 'prior_violent_arrest', 'prior_violent_indictment', 'prior_violent_conviction',
                 'prior_felony_arrest', 'prior_felony_indictment', 'prior_felony_conviction']
prior_def = np.nan
counter = 0 
for i, row in flag_df.iterrows():
    current_def = row.cms_defendant_guid
    
    if current_def != prior_def:
        priors = [0,0,0,0,0,0,0,0,0]   
    
    flag_df.loc[i, priors_labels] = np.where(priors, True, False)

    priors = check_priors(priors, row)
    prior_def = current_def
    
    if i%10000==0: print()
    
    counter+=1
    if counter==1000: break

KeyboardInterrupt: 

In [ ]:
%%time
merge_df = flag_df[['CrimeCase', 'prior_arrest', 'prior_indictment',
       'prior_conviction', 'prior_violent_arrest', 'prior_violent_indictment',
       'prior_violent_conviction', 'prior_felony_arrest',
       'prior_felony_indictment', 'prior_felony_conviction']]
final_df = df.merge(merge_df, how='left', on='CrimeCase')
final_df.to_csv(f'stanford_data_pull_final_{date.today()}')

In [ ]:
final_df.head(100)

In [ ]:
final_df.shape

In [51]:
%%time
priors_labels = ['prior_arrest', 'prior_indictment', 'prior_conviction',
                 'prior_violent_arrest', 'prior_violent_indictment', 'prior_violent_conviction',
                 'prior_felony_arrest', 'prior_felony_indictment', 'prior_felony_conviction']
flag_df[priors_labels].head()
priors=[0,1,0,0,1,1,0,0,1]
flag_df.loc[0, priors_labels] = np.where(priors, True, False)
flag_df[priors_labels].head()

CPU times: total: 328 ms
Wall time: 328 ms


,prior_arrest,prior_indictment,prior_conviction,prior_violent_arrest,prior_violent_indictment,prior_violent_conviction,prior_felony_arrest,prior_felony_indictment,prior_felony_conviction
0,False,True,False,False,True,True,False,False,True
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [43]:
ids = flag_df.prior_arrest.isna()

In [ ]:
ids